In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

if '/home/jovyan/' not in sys.path:
    sys.path.append('/home/jovyan/')

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src import buildXyzMapCommand
from src import processScansCommand


data_dir = '/home/jovyan/SharedData'


from utils.imutil import *
from utils.progress import *

import matplotlib.pyplot as plt
%matplotlib inline




In [ ]:
scan = 'scan-0630T12-26-48'
blur_distance = 0
normal_h, inverse_h, normal_v, inverse_v, reference_image, cam_mask_image, min_image = processScansCommand.load_scan(
            scan, data_dir, blur_distance)

In [ ]:
imshow(min_image, 'jpg', zoom=0.1)

In [ ]:
diff_code_h, diff_code_v, confidence = processScansCommand.calculate_diference(
            normal_h, inverse_h, normal_v, inverse_v, cam_mask_image)

In [ ]:
# Render the confidence
plt.hist(confidence.flatten(), bins=64, range=(0, 1))
imshow(confidence*255, 'jpg', zoom=0.2)
# plt.hist(confidence, 10)


In [ ]:
packed_h, packed_v = processScansCommand.pack_image(diff_code_h, diff_code_v, data_dir)



In [ ]:
# Render the cam map
cam_map = processScansCommand.add_channel(np.dstack((packed_h, packed_v))).astype(np.float32)
cam_map[confidence < 0.2] = 0
cam_map[:,:,0] /= cam_map[:,:,0].max()
cam_map[:,:,1] /= cam_map[:,:,1].max()

imshow(cam_map*255, 'jpg')#, zoom=0.3)

In [ ]:
projector_size = processScansCommand.get_projector_size(data_dir)
pro_map, pro_confidence = processScansCommand.build_pro_map(
            packed_v, packed_h, confidence, projector_size[0], projector_size[1])

In [ ]:
# Render the pro map
debug_pro_map = pro_map.copy().astype(np.float32)
debug_pro_map[:,:,0] = debug_pro_map[:,:,0] / debug_pro_map[:,:,0].max()
debug_pro_map[:,:,1] = debug_pro_map[:,:,1] / debug_pro_map[:,:,1].max()

imshow(debug_pro_map * 255, 'jpg', zoom=0.3)

debug_pro_map[pro_confidence < 0.3] = 0
imshow(debug_pro_map * 255, 'jpg', zoom=0.3)



## Debug single pixel

In [ ]:

# Good pixel (reflection):
pixel = (1669,673) # y,x

# Bad pixel (non-reflection)
# pixel = (1677,632) # y,x
# Bad pixel (overlapping)
# pixel = (1677, 3975)

print("Confidence ", confidence[pixel[0], pixel[1]])
print("Projector coordinate ", packed_v[pixel[0], pixel[1]],',', packed_h[pixel[0], pixel[1]])

print()
print("Diff codes")

# plt.hist(diff_code_h[:,pixel[0], pixel[1]], bins=20, range=(-200,200))
diff_pixel = diff_code_h[:,pixel[0], pixel[1]]
plt.figure(0)
plt.plot(np.arange(len(diff_pixel)),diff_pixel)
plt.axhline(y=0, color='r', linestyle='dashed')

plt.figure(1)
diff_pixel = diff_code_v[:,pixel[0], pixel[1]]
plt.plot(np.arange(len(diff_pixel)),diff_pixel)
plt.axhline(y=0, color='r', linestyle='dashed')

assert packed_v[pixel[0], pixel[1]] == 2402
assert packed_h[pixel[0], pixel[1]] == 386
# plt.hist(diff_code_v[:,pixel[0], pixel[1]])

In [ ]:
# Visualize normal/inverse

diff_pixel = normal_h[:,0,pixel[0], pixel[1]]
plt.figure(0)
plt.plot(np.arange(len(diff_pixel)),diff_pixel)

diff_pixel = inverse_h[:,0,pixel[0], pixel[1]]
plt.plot(np.arange(len(diff_pixel)),diff_pixel)

diff_pixel = normal_v[:,0,pixel[0], pixel[1]]
plt.figure(1)
plt.plot(np.arange(len(diff_pixel)),diff_pixel)

diff_pixel = inverse_v[:,0,pixel[0], pixel[1]]
plt.plot(np.arange(len(diff_pixel)),diff_pixel)



# Visualize graycode LUT

In [ ]:
lut = processScansCommand.load_lut(data_dir, 2)
# decoded = processScansCommand.decode_graycode(lut, np.asarray([[[1,0]]]))
# lut.dot(1 << np.arange(lut.size)[::-1])
print(lut.shape)
lut
# lut.dot(2**np.arange(lut.shape[1]-1,-1,-1))


In [ ]:
data= np.asarray([
    [
        [0,0], [1,1], [0,0]
    ],
    [
        [1,0], [1,0], [1,0]
    ]
])

data = np.moveaxis(data, -1, 0)

# data = np.packbits(data, axis=2)
data = processScansCommand.pack_raw(data)
print(data)
print(data.shape)
print(lut)
# print(lut[data])
# data == lut
# print( (data == lut))
lookup = np.equal(data, lut).nonzero()
print(lookup)
# data[:,:,] = (data == lut).nonzero()[2]
# res
print(lookup)

res = np.zeros((data.shape[0], data.shape[1]), dtype=np.uint8)
res[lookup[0], lookup[1]] = lookup[2]

res

# np.packbits([1,1,1,0,1,0])

In [ ]:
n = 5
print(processScansCommand.gray_lut(n).astype(np.uint8))
print(processScansCommand.load_lut(data_dir,n))

